### Описание задания:

Решить задачу регрессии используя деревья решений. В качестве датасета необходимо взять данные о недвижимости Калифорнии из библиотеки sklearn.datasets. Целевая переменная – MedHouseVal. Прочитать информацию о признаках датасета можно, выполнив следующий код – print(fetch_california_housing().DESCR). На полученных данных построить модель регрессии и дерево решений.

### Этапы работы:

1) Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую).

2) Проведите первичный анализ.
**a.** Проверьте данные на пропуски. Удалите в случае обнаружения.
**b.** Нормализуйте один из признаков.

3) Разделите выборку на обучающее и тестовое подмножества. 80% данных оставить на обучающее множество, 20% - на тестовое.

4) Обучите модель регрессии на обучающем множестве.

5) Для тестового множества предскажите целевую переменную и сравните с истинным значением, посчитав точность предсказания модели. Для этого используйте встроенную функцию score.

6) Обучите дерево решений на обучающем множестве.
**a.** Повторите п. 5 для полученной модели.
**b.** Визуализируйте часть дерева решений. Убедитесь, что график получился читабельным. Посмотрите примеры визуализации по ссылке.

7) Оптимизируйте глубину дерева (max_depth). *Оптимизируйте ещё один параметр модели на выбор.
**a.** Повторите п. 5 для полученной модели.

8) Сформулируйте выводы по проделанной работе.
**a.** Сравните точность двух моделей.
**b.** Напишите свое мнение, для каких задач предпочтительнее использовать обученные в работе модели? Какие у них есть плюсы и минусы?